## 1: Import required packages & functions <a class="anchor" id="chapter1"></a>

In [ ]:
print('Importing libraries')

%reset -f

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# clear all variables
for i in list(globals().keys()):
    if(i[0] != '_'):
        exec('del {}'.format(i))

#suppress future warnings -- not really a good idea 
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# import required dependencies
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
from keras.models import Model
from keras.layers import Input, Dense
from numpy.random import seed
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, silhouette_score
import csv
import xlrd
import math
from matplotlib import figure
from IPython.display import FileLink, FileLinks
from IPython.display import set_matplotlib_formats
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict, StratifiedKFold 
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

#import utils
#from utils import encode_features_v3

set_matplotlib_formats('png', 'pdf') # uses vector figures in pdf exports --
plt.style.use('seaborn-pastel')

# create a folder
def create_folder(folder):
    import os     
    try: 
        os.mkdir(folder) 
    except FileExistsError:
        print("Directory [ %s ] already exists"%folder)

## 2: Load data   <a class="anchor" id="chapter2"></a>

In [ ]:
create_folder("Misclassified")

dataset = pd.read_csv('Classification/new_balanced_no_AB.csv');
dataset = dataset.drop(columns=['Case'])
X = dataset.iloc[:,:-1]
y = np.ravel(dataset.iloc[:,-1])

## 3: Function to predict for all classifiers   <a class="anchor" id="chapter3"></a>

In [ ]:
logreg_fscore_mean=[]; logreg_precision_mean=[]; logreg_recall_mean=[]; logreg_balanced_acc_mean=[]
logreg_sens_mean=[]; logreg_spec_mean=[]; logreg_acc_mean=[]; 
logreg_average_precision_mean=[]; logreg_auc_mean=[];

dt_fscore_mean=[]; dt_precision_mean=[]; dt_recall_mean=[]; dt_balanced_acc_mean=[]
dt_sens_mean=[]; dt_spec_mean=[]; dt_acc_mean=[]; 
dt_average_precision_mean=[]; dt_auc_mean=[]

knn_fscore_mean=[]; knn_precision_mean=[]; knn_recall_mean=[]; knn_balanced_acc_mean=[]
knn_sens_mean=[]; knn_spec_mean=[]; knn_acc_mean=[]; 
knn_average_precision_mean=[]; knn_auc_mean=[]

lda_fscore_mean=[]; lda_precision_mean=[]; lda_recall_mean=[]; lda_balanced_acc_mean=[]
lda_sens_mean=[]; lda_spec_mean=[]; lda_acc_mean=[]; 
lda_average_precision_mean=[]; lda_auc_mean=[]

gnb_fscore_mean=[]; gnb_precision_mean=[]; gnb_recall_mean=[]; gnb_balanced_acc_mean=[]
gnb_sens_mean=[]; gnb_spec_mean=[]; gnb_acc_mean=[]; 
gnb_average_precision_mean=[]; gnb_auc_mean=[]

svm_rbf_fscore_mean=[]; svm_rbf_precision_mean=[]; svm_rbf_recall_mean=[]; svm_rbf_balanced_acc_mean=[] 
svm_rbf_sens_mean=[]; svm_rbf_spec_mean=[]; svm_rbf_acc_mean=[]; 
svm_rbf_average_precision_mean=[]; svm_rbf_auc_mean=[]

svm_linear_fscore_mean=[]; svm_linear_precision_mean=[]; svm_linear_recall_mean=[]; svm_linear_balanced_acc_mean=[] 
svm_linear_sens_mean=[]; svm_linear_spec_mean=[]; svm_linear_acc_mean=[];
svm_linear_average_precision_mean=[]; svm_linear_auc_mean=[];


In [ ]:
all_y_pred = pd.DataFrame()

logreg_y_pred = pd.DataFrame(); dt_y_pred = pd.DataFrame(); knn_y_pred = pd.DataFrame(); lda_y_pred = pd.DataFrame();
gnb_y_pred = pd.DataFrame(); svm_rbf_y_pred = pd.DataFrame(); svm_linear_y_pred = pd.DataFrame();

# create loocv procedure
cv = LeaveOneOut()
print("subset:")
for num in range(0,22):
    print(num," \b", end="")
    
    #get data and target
    X_subset = X.iloc[:,num::-1]
    
    # feature scaling
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_subset)
    
    logreg = LogisticRegression(random_state = 220)
    dt = DecisionTreeClassifier(random_state = 220)
    knn = KNeighborsClassifier()
    lda = LinearDiscriminantAnalysis()
    gnb = GaussianNB()
    svm_rbf = SVC(kernel='rbf', random_state = 220)
    svm_linear = SVC(kernel='linear', random_state = 220)
    
    # calculate cross validation score for each of the fitted models 
    for model in ['logreg', 'dt', 'knn', 'lda', 'gnb', 'svm_rbf', 'svm_linear']:   
        # enumerate splits
        y_true, y_pred = list(), list()
        for train_ix, test_ix in cv.split(X_scaled):
            # split data
            X_train, X_test = X_scaled[train_ix, :], X_scaled[test_ix, :]
            y_train, y_test = y[train_ix], y[test_ix]
            # fit model
            exec("%s"%model+".fit(X_train, y_train)")
            # evaluate model
            exec("yhat = %s"%model+".predict(X_test)")
            # store
            y_true.append(y_test[0])
            y_pred.append(yhat[0])
        exec("%s"%model+"_y_pred['%s"%model+"_%d"%(num+1)+"'] = y_pred")
        
        exec("%s"%model+"_fscore_mean.append(f1_score(y, y_pred))") 
        exec("%s"%model+"_precision_mean.append(precision_score(y, y_pred))")
        exec("%s"%model+"_recall_mean.append(recall_score(y, y_pred))")
        exec("%s"%model+"_balanced_acc_mean.append(balanced_accuracy_score(y, y_pred))")
        exec("%s"%model+"_acc_mean.append(accuracy_score(y, y_pred))")
        exec("%s"%model+"_average_precision_mean.append(average_precision_score(y, y_pred))")
#         exec("sens, spec, auc_value = perf_eval(y, y_pred)")
#         exec("%s"%model+"_sens_mean.append(sens)") 
#         exec("%s"%model+"_spec_mean.append(spec)") 
#         exec("%s"%model+"_auc_mean.append(auc_value)")



## 4: Binary Table Matrix   <a class="anchor" id="chapter4"></a>

In [ ]:
y_pred = pd.DataFrame()
all_y_pred = pd.DataFrame()
for model in ['logreg', 'dt', 'knn', 'lda', 'gnb', 'svm_rbf', 'svm_linear']:
    exec("all_y_pred = pd.concat([all_y_pred, %s"%model+"_y_pred], axis=1)")

In [ ]:
#------------------ FUNCTION -----------------------#
def binary_table(y_pred, y_true):
    data = pd.DataFrame(y_pred)
    data['y_true'] = y_true['dem_nver4']

    cond_1 = (data.iloc[:,1] == 1) & (data.iloc[:,0] == 1)
    cond_2 = (data.iloc[:,1] == 0) & (data.iloc[:,0] == 0)
    cond_3 = (data.iloc[:,1] == 1) & (data.iloc[:,0] != 1)
    cond_4 = (data.iloc[:,1] == 0) & (data.iloc[:,0] != 0)

    conditions = [cond_1,cond_2,cond_3,cond_4]
#     choices = [0,0,1,-1]
    choices = [0,0,1,-1]

    data.drop('y_true', axis=1, inplace=True)
    data.iloc[:,0] = np.select(conditions, choices, default='null')
    return data
#---------------------------------------------------#

X = pd.DataFrame(dataset.iloc[:,:-1])
y = pd.DataFrame(dataset.dem_nver4)

all_binary_result = pd.DataFrame(y)

for classifier in ['logreg','dt','knn','lda','gnb','svm_rbf','svm_linear']:
    for subset in np.arange(1, 23, 1):
        exec("%s"%classifier+"_%s"%subset+" = binary_table(all_y_pred['%s"%classifier+"_%s"%subset+"'],y)")
        exec("all_binary_result['%s"%classifier+"_%s"%subset+"'] = %s"%classifier+"_%s"%subset)

In [ ]:
from pandas.plotting import table 
all_binary_result.drop('dem_nver4', axis=1,inplace=True)

aa = all_binary_result.style.applymap(lambda x: "background-color: red" if x == '-1' else "background-color: blue" if x == '1' else "background-color: white")

all_binary_result = all_binary_result.astype(int)
aaa_test = all_binary_result

## 5: Hierarchical Clustering   <a class="anchor" id="chapter5"></a>

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

hc = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage ='average')

y_hc=hc.fit(all_binary_result)

#get rows' clusters
membership = hc.labels_
binary_clusters = pd.DataFrame(all_binary_result).copy()
binary_clusters['clusters'] = membership


fig=plt.figure(figsize=(14, 10)) 
dendrogram = sch.dendrogram(sch.linkage(all_binary_result, method  = "average"))
plt.title('Dendrogram')
plt.xlabel('Samples/Patients')
plt.ylabel('Euclidean distances')
plt.axhline(y=14, color='r', linestyle='--')
plt.show()

In [ ]:
# save membership to csv file for clinical signatures
membership_value = pd.DataFrame()
membership_value['membership'] = membership
membership_value.to_csv(r'Classification/membership_value.csv', index=False, mode = 'w')


dataset_membership = pd.read_csv('Classification/new_balanced_no_AB.csv');
dataset_membership = dataset_membership.drop(columns=['Case'])
dataset_membership['membership'] = membership
dataset_membership.to_csv(r'Classification/dataset_membership.csv', index=False, mode = 'w')


In [ ]:
import seaborn as sns; sns.set(color_codes=True)
heatmap_binary = all_binary_result
heatmap_binary.columns = pd.MultiIndex.from_product([['logreg', 'dt', 'knn', 'lda', 'gnb', 'svm_rbf', 'svm_linear'], 
                                                     np.arange(1, 23, 1)], 
                                                   names=['Classifiers', 'Subsets of Features'])
# ### For tree colors
clusters = binary_clusters.pop("clusters")

#### [0 - FN - 22] [1 - FP - 24] [2 - TP&TN - 68]

In [ ]:
clusters.value_counts()

## 6: Cluster Heatmap  <a class="anchor" id="chapter6"></a>

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
#### classifiers level
classi_labels = heatmap_binary.columns.get_level_values("Classifiers")
classi_pal = sns.cubehelix_palette(classi_labels.unique().size, light=.9, dark=.1, reverse=True, start=1, rot=-2)
classi_lut = dict(zip(map(str, classi_labels.unique()), classi_pal))

#### Create index using the columns for classifiers
classi_colors = pd.Series(classi_labels, index=heatmap_binary.columns).map(classi_lut)

#### subsets level
subset_labels = heatmap_binary.columns.get_level_values("Subsets of Features")
subset_pal = sns.cubehelix_palette(subset_labels.unique().size, as_cmap=True, hue=0, start=0.3, light=1.0, dark=0.0,rot=-10.0, gamma=10.0)

#### Create index using the columns for subsets
subset_colors = pd.Series(subset_labels, index=heatmap_binary.columns).map(subset_pal)

# classi_subset_colors = pd.DataFrame(subset_colors)
classi_subset_colors = pd.DataFrame(subset_colors)#pd.DataFrame(classi_colors).join(pd.DataFrame(subset_colors))

### decoding colors for heatmap and dendograms
red=(0.5, 0.0, 0.0, 1.0)
blue=(0.0, 0.0, 0.5, 1.0)
black=(0.0, 0.0, 0.0, 0.0)
white=(1,1,1,1)
grey=(0.5, 0.5, 0.5, 1.0)

### tree clusters colors

clusters_lut = dict(zip(clusters.unique(), [grey, red, blue]))
row_colors = clusters.map(clusters_lut)
# df_colors = pd.DataFrame(data={'False Positive [18 Individuals]': row_colors[row_colors == blue], 'False Negative [22 Individuals]': row_colors[row_colors == red], 'Correctly Classified [62 Individuals]': row_colors[row_colors == grey]})

''''''''''''''''''
plt.rc('font', size=14)          # controls default text sizes
plt.rc('axes', titlesize=14)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('ytick', labelsize=4)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)    # legend fontsize
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels

plt.rcParams['savefig.facecolor']='white'

myColors = (blue,black, red)
cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))


# color bar keyword arguments
cbar_kws = {"shrink":1,
            'ticks': ['f',0,-1], # set ticks of color bar
            'label':'Color Bar'}

g = sns.clustermap(all_binary_result,
                    method= 'average',
                    metric= "euclidean",
                    col_cluster=False,
                    row_cluster=True,
                    col_colors = classi_subset_colors,
                    row_colors=row_colors,
                    cmap=cmap,
                    tree_kws=dict(linewidths=0.95, colors=([red]*21+[blue]*23+[grey]*67+[(0,0,0,1)]*2)),
                    linewidths=0.0,
                   rasterized=True,
                   xticklabels=False, 
                   yticklabels=False, 
                   center=0) 

g.cax.set_visible(False)
g.ax_row_dendrogram.set_visible(True)
g.ax_col_dendrogram.set_visible(True)

ax = g.ax_heatmap
ax.set_xlabel("")
ax.set_ylabel("Individuals")
plt.tight_layout()

plt.show()

g.savefig('Figures/all_subsets_clustermap.pdf',dpi=300, bbox_inches="tight")
g.savefig('Figures/all_subsets_clustermap.png',dpi=300, bbox_inches="tight")


## 7: Prepare Clinical Features for Signature  <a class="anchor" id="chapter7"></a>

In [ ]:
master = pd.read_csv('../data/master.csv',header = 0)
clinical_features = pd.read_csv('../data/clinical features.csv',header = 0)

aa = pd.read_csv('Classification/dataset_membership.csv')

new_balanced_no_AB = pd.read_csv('Classification/new_balanced_no_AB.csv')
selected_clinical = new_balanced_no_AB.iloc[:,0:1].merge(master[clinical_features.feature], how = 'inner', on = ['Case'])
selected_clinical

''' Adjusment of brain weight based on gender '''
women_braiwgt = selected_clinical[selected_clinical.sex == 'Women']
men_braiwgt = selected_clinical[selected_clinical.sex == 'Men']
gender_missing = selected_clinical[selected_clinical.sex.isnull()]

#Calculate z-score by mean
from scipy import stats
women_braiwgt_zscore = stats.zscore(women_braiwgt['brain weight'], nan_policy='omit')
men_braiwgt_zscore = stats.zscore(men_braiwgt['brain weight'], nan_policy='omit')
women_braiwgt['brain weight'] = women_braiwgt_zscore
men_braiwgt['brain weight'] = men_braiwgt_zscore

selected_clinical = pd.concat([women_braiwgt, men_braiwgt])
selected_clinical = pd.concat([selected_clinical, gender_missing])
selected_clinical.sort_values(by='Case', inplace=True)

membership_value = pd.read_csv('Classification/membership_value.csv',header = 0) # read clinical dataset
selected_clinical['membership_value'] = membership_value.membership.copy()
values = pd.DataFrame(membership_value.membership).copy()
selected_clinical['membership'] = values.membership.replace({0:'TP&TN', 1:'FN', 2:'FP'})

'''
        This file is used for clinical signatures by (https://github.com/emmanueljammeh/cfas)
        
'''
selected_clinical.to_csv(r'Classification/clinical_with_memberships.csv',mode = 'w',index=False)

selected_clinical



## 8: Clinical Signature Figure  <a class="anchor" id="chapter8"></a>

From Emmanuel A. Jammeh code results, the file shows the features with their coefficient values that are having association with the three clusters (FP, FN, and TP&TN)

In [ ]:
clinical_singnature = pd.read_csv('../data/relevant_clinical_signatures_coefficients.csv',header = 0)
clinical_singnature['Sum'] = clinical_singnature.iloc[:,2:5].abs().sum(axis=1)
clinical_singnature.sort_values(by='Sum', inplace=True, ascending=False)
clinical_singnature

red=(0.5, 0.0, 0.0, 1.0)
blue=(0.0, 0.0, 0.5, 1.0)
grey=(0.5, 0.5, 0.5, 1.0)
%config InlineBackend.figure_format ='retina'
sns.set_style("whitegrid", {'axes.grid' : False})
# plt.subplots_adjust(wspace=0.10)
fig= plt.figure(figsize=(10,8))
size = 16
plt.rc('font', size=size)          # controls default text sizes
plt.rc('axes', titlesize=size)     # fontsize of the axes title
plt.rc('axes', labelsize=size)    # fontsize of the x and y labels
plt.rc('ytick', labelsize=size)    # fontsize of the tick labels
plt.rc('legend', fontsize=size)    # legend fontsize
plt.rc('xtick', labelsize=size)    # fontsize of the tick labels
ax = plt.subplot(3,1,1)
ax.bar(clinical_singnature.features, clinical_singnature.Cluster1, width=0.7, color=red)
ax.axhline(y=0, color='k')
plt.ylabel('Coefficient')
plt.xticks(' ')
plt.title('False Negative')
plt.ylim(-1, 1,0.25)
plt.tight_layout()

ax = plt.subplot(3,1,2)
ax.bar(clinical_singnature.features, clinical_singnature.Cluster2, width=0.7, color=blue)
ax.axhline(y=0, color='k')
plt.ylabel('Coefficient')
plt.xticks(' ')
plt.title('False Positive')
plt.ylim(-1, 1)
plt.tight_layout()

ax = plt.subplot(3,1,3)
ax.bar(clinical_singnature.features, clinical_singnature.Cluster0, width=0.7, color=grey)
plt.ylabel('Coefficient')
plt.title('Correctly Classified')
ax.axhline(y=0, color='k')
plt.tight_layout()
plt.xticks(rotation=45, horizontalalignment="right")
plt.ylim(-1, 1)

plt.savefig('Figures/clinical_signature.png', dpi=300, bbox_inches='tight')
plt.savefig('Figures/clinical_signature.pdf', dpi=300, bbox_inches='tight')